In [1]:
import sys
import mysql.connector
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import os
from dotenv import load_dotenv
load_dotenv()
#ESTE ARGUMENTO NO SE USA EN ESTE REPORTE, SERÁ 0 SIEMPRE UWU
id=0
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')

a_color='#354F84'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
#Seccion para traer informacion de la base
query = ('SELECT * from customers where id =1')

# join para cobros
# cobros=pd.read_sql('Select cobros.* ,customers.customer,internal_orders.invoice, users.name from ((cobros inner join internal_orders on internal_orders.id = cobros.order_id) inner join customers on customers.id = internal_orders.customer_id )inner join users on cobros.capturo=users.id',cnx)


#traer datos de los pedidos
pedidos=pd.read_sql("""Select internal_orders.* ,customers.clave,customers.alias,
coins.exchange_sell, coins.coin, coins.symbol,coins.code
from ((
    internal_orders
    inner join customers on customers.id = internal_orders.customer_id )
    inner join coins on internal_orders.coin_id = coins.id)
     """,cnx)
clientes=pd.read_sql("select * from customers",cnx)
cobros=pd.read_sql("""select cobro_orders.*,internal_orders.coin_id as coin_pedido,internal_orders.customer_id
                     from (((
                         cobro_orders 
    inner join cobros on cobros.id=cobro_orders.cobro_id)
    inner join internal_orders on internal_orders.id = cobros.order_id )
    inner join coins on internal_orders.coin_id = coins.id) """,cnx)

facturas=pd.read_sql("""select factures.*,internal_orders.coin_id as coin_pedido,internal_orders.customer_id  
                     from (
                         factures
    inner join internal_orders on internal_orders.id = factures.order_id )""",cnx)
creditos=pd.read_sql("""select credit_notes.*,internal_orders.coin_id as coin_pedido,internal_orders.customer_id as customer_pedido 
                     from (
                         credit_notes
    inner join internal_orders on internal_orders.id = credit_notes.order_id ) """,cnx)
print(creditos)
nordenes=len(pedidos)
df=pedidos[['date']]
print(cobros['order_id'])
tc=pd.read_sql('select * from coins where id=13 ',cnx)['exchange_sell'].values[0]
#Filtrando datos---------------------------------------
#Separar pedidos vigentes
    #Agregar columna con saldo en los cobros
pedidos=pedidos.assign(saldo=0.0)
for i in range(len(pedidos)):
    pedidos['saldo'].values[i]=cobros.loc[cobros['order_id']==pedidos['id'].values[i],'amount'].sum()
pedidos=pedidos.loc[pedidos['total']-pedidos['saldo']>1]
writer = pd.ExcelWriter('storage/report/CxC_cliente_desglosado_vigente'+str(id)+'.xlsx', engine='xlsxwriter')

workbook = writer.book
##FORMATOS PARA EL TITULO------------------------------------------------------------------------------
rojo_l = workbook.add_format({
    'bold': 0,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    #'fg_color': 'yellow',
    'font_color': 'red',
    'font_size':16})
negro_s = workbook.add_format({
    'bold': 0,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':12})
negro_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':13}) 
rojo_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'red',
    'font_size':13})      

#FORMATOS PARA CABECERAS DE TABLA --------------------------------
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'center',
    'fg_color': 'yellow',
    'border': 1,})

blue_header_format = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1})
blue_header_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'font_size':13})


#FORMATOS PARA TABLAS PER CE------------------------------------

blue_content = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})

blue_content_bold = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11,
    'border_color':a_color,
    'num_format': '[$$-409]#,##0.00'})

blue_content_bold_dll = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11,
    'bg_color': '#b4e3b1',
    'border_color':a_color,
    'num_format': '[$$-409]#,##0.00'})
blue_content_dll = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'bg_color': '#b4e3b1',
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})
blue_content_date = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':9,
    'border_color':a_color,
    'num_format': 'dd/mm/yyyy'})
#FOOTER FORMATS---------------------------------------------------------
observaciones_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color':'#BDD7EE',
    'border': 1})

total_cereza_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color':'#F4B084',
    'border': 1})


df[0:1].to_excel(writer, sheet_name='Sheet1', startrow=7,startcol=6, header=False, index=False)
worksheet = writer.sheets['Sheet1']
#Encabezado del documento--------------

import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y")
df[0:1].to_excel(writer, sheet_name='Sheet1', startrow=7,startcol=6, header=False, index=False)
worksheet = writer.sheets['Sheet1']
#Encabezado del documento--------------
worksheet.merge_range('B2:F2', 'CUENTAS POR COBRAR REPORTE 3B ', negro_b)
worksheet.merge_range('B3:F3', 'DERECHOS ADQUIRIDOS POR COBRAR VIGENTES', negro_s)
worksheet.merge_range('B4:F4', 'CLASIFICADAS POR CLIENTE DESGLOSADO', negro_b)

worksheet.write('H2', 'AÑO', negro_b)

worksheet.write('I2', year, negro_b)
worksheet.merge_range('J2:K3', """FECHA DEL REPORTE
DD/MM/AAAA""", negro_b)

worksheet.write('L2', date, negro_b)
worksheet.insert_image("A1", "img/logo/logo.png",{"x_scale": 0.6, "y_scale": 0.6})
#llenando la tabla
row_index=7
for i in range(0,len(clientes)):
   print(str(i)+' esa no')
   if(len(pedidos.loc[pedidos['customer_id']==clientes['id'].values[i]])>0):
        print(str(i)+'esta si')
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        worksheet.merge_range('H'+str(4+row_index)+':R'+str(4+row_index), 'CUENTAS POR COBRAR POR CLIENTE DESGLOSADO', negro_b)

        worksheet.merge_range('C'+str(6+row_index)+':C'+str(10+row_index), 'PDA', blue_header_format)
        worksheet.merge_range('D'+str(6+row_index)+':D'+str(10+row_index), 'PI CANTIDAD', blue_header_format)
        worksheet.merge_range('E'+str(6+row_index)+':E'+str(10+row_index), 'FECHA', blue_header_format)

        worksheet.merge_range('F'+str(6+row_index)+':G'+str(9+row_index), 'CLIENTE', blue_header_format)
        worksheet.write('F'+str(10+row_index), 'NUMERO', blue_header_format)
        worksheet.write('G'+str(10+row_index), 'NOMBRE CORTO', blue_header_format)

        worksheet.merge_range('H'+str(6+row_index)+':H'+str(10+row_index), """ 
                                  MONEDA""", blue_header_format)

        worksheet.merge_range('I'+str(6+row_index)+':L'+str(6+row_index), 'DERECHOS ADQUIRIDOS', blue_header_format)
        worksheet.merge_range('I'+str(7+row_index)+':J'+str(9+row_index), """IMPORTE TOTAL 
(DERECHOS ADQUIRIDOS) 
SIN IVA""", blue_header_format)
        worksheet.write('I'+str(10+row_index), 'MN', blue_header_format)
        worksheet.write('J'+str(10+row_index), 'DLLS', blue_header_format)


        worksheet.merge_range('K'+str(7+row_index)+':L'+str(9+row_index), """COBRADO
(IMPORTE POR COBRAR)
SIN IVA""", blue_header_format)
        worksheet.write('K'+str(10+row_index), 'MN', blue_header_format)
        worksheet.write('L'+str(10+row_index), 'DLLS', blue_header_format)


    

        worksheet.merge_range('M'+str(7+row_index)+':M'+str(10+row_index), '% POR COBRAR DEL PEDIDO INTERNO', blue_header_format)

        worksheet.merge_range('N'+str(6+row_index)+':O'+str(6+row_index), 'DERECHOS ADQUIRIDOS POR COBRAR CONTABLES', blue_header_format)


        worksheet.merge_range('N'+str(7+row_index)+':O'+str(9+row_index), """POR FACTURAR
CXC CONTABLES
(SIN IVA)""", blue_header_format)
        worksheet.write('N'+str(10+row_index), 'MN', blue_header_format)
        worksheet.write('O'+str(10+row_index), 'DLLS', blue_header_format)
        
        worksheet.merge_range('P'+str(6+row_index)+':P'+str(10+row_index), 'STATUS', blue_header_format)
        row_index=str(11+row_index)
        
        print(cobros.columns)
        cobros_mn=cobros.loc[(cobros['customer_id']==clientes['id'].values[i])&(cobros['coin_pedido']==1)]
        facturas_mn=facturas.loc[(facturas['customer_id']==clientes['id'].values[i])&(facturas['coin_pedido']==1)]
        notas_mn=creditos.loc[(creditos['customer_pedido']==clientes['id'].values[i])&(creditos['coin_pedido']==1)]
        
        cobros_dlls=cobros.loc[(cobros['customer_id']==clientes['id'].values[i])&(cobros['coin_pedido']!=1)]
        facturas_dlls=facturas.loc[(facturas['customer_id']==clientes['id'].values[i])&(facturas['coin_pedido']!=1)]
        notas_dlls=creditos.loc[(creditos['customer_id']==clientes['id'].values[i])&(creditos['coin_pedido']!=1)]
        
        pedidos_mn=pedidos.loc[(pedidos['customer_id']==clientes['id'].values[i])&(pedidos['coin_id']==1)]
        pedidos_dlls=pedidos.loc[(pedidos['customer_id']==clientes['id'].values[i])&(pedidos['coin_id']!=1)]
        
        this_pedidos=pedidos.loc[(pedidos['customer_id']==clientes['id'].values[i])]
        
        total_mn=  pedidos_mn['total'].sum()/1.16
        total_dlls=pedidos_dlls['total'].sum()/1.16
        #datos generales del pedido
        #worksheet.write('B'+row_index, str(pedidos['noha'].values[i]), blue_content)
        for k in range(len(this_pedidos)):
            
            this_cobros=cobros.loc[(cobros['order_id']==this_pedidos['id'].values[k])]
            this_facturas=facturas.loc[(facturas['order_id']==this_pedidos['id'].values[k])]
            this_notas=creditos.loc[(creditos['order_id']==this_pedidos['id'].values[k])]
        
            worksheet.write('C'+row_index, str(k+1), blue_content)
            worksheet.write('D'+row_index, this_pedidos['invoice'].values[k], blue_content)
            worksheet.write('E'+row_index, this_pedidos['reg_date'].values[k], blue_content)
            worksheet.write('F'+row_index, str(clientes['clave'].values[i]), blue_content)
            worksheet.write('G'+row_index, str(clientes['alias'].values[i]), blue_content)
            worksheet.write('H'+row_index, this_pedidos['code'].values[k], blue_content)
            #subtotal
            #pesos
            if(this_pedidos['coin_id'].values[k]==1):
                worksheet.write('I'+row_index,this_pedidos['total'].values[k]/1.16, blue_content)
                worksheet.write('J'+row_index, 0, blue_content_dll)
                #por cobrar
                worksheet.write('K'+row_index,this_pedidos['total'].values[k]/1.16- this_cobros['amount'].sum()/1.16 , blue_content)
                worksheet.write('L'+row_index, 0 , blue_content_dll)
                #facturado
                xc=this_pedidos['total'].values[k]/1.16- this_cobros['amount'].sum()/1.16
                if(xc>1):
                    #por facturar
                    worksheet.write('N'+row_index,xc-this_facturas['amount'].sum()/1.16-this_notas['amount'].sum()/1.16-this_cobros['amount'].sum()/1.16 , blue_content)
                    worksheet.write('O'+row_index, 0 , blue_content_dll)
                else:
                    #por facturar
                    worksheet.write('N'+row_index,0, blue_content)
                    worksheet.write('O'+row_index, 0 , blue_content_dll)
           
            
            else:
                worksheet.write('I'+row_index,0, blue_content)
                worksheet.write('J'+row_index, this_pedidos['total'].values[k]/1.16, blue_content_dll)
               #por cobrar
                worksheet.write('K'+row_index,0 , blue_content)
                worksheet.write('L'+row_index, this_pedidos['total'].values[k]/1.16- this_cobros['amount'].sum()/1.16 , blue_content_dll)
         #facturado
                xc=this_pedidos['total'].values[k]/1.16- this_cobros['amount'].sum()/1.16
                if(xc>1):
                    #por facturar
                    worksheet.write('N'+row_index, 0, blue_content)
                    worksheet.write('O'+row_index, xc-this_facturas['amount'].sum()/1.16-this_notas['amount'].sum()/1.16-this_cobros['amount'].sum()/1.16 , blue_content_dll)
                else:
                    #por facturar
                    worksheet.write('N'+row_index,0, blue_content)
                    worksheet.write('O'+row_index, 0 , blue_content_dll)
           
            
            worksheet.write('M'+row_index, "{:.2f}".format((this_pedidos['total'].values[k]- this_cobros['amount'].sum())*100/this_pedidos['total'].values[k])+"%", blue_content)
            
            
             #status
            if(total_mn+total_dlls-cobros_mn['amount'].sum()/1.16-cobros_dlls['amount'].sum()/1.16>1):
                worksheet.write('P'+row_index,'ACTIVO', blue_content)
            else:
                worksheet.write('P'+row_index,'CERRADO', blue_content)
            row_index=str(int(row_index)+1)
        trow=int(row_index)

        worksheet.write('H'+str(trow), 'Subtotales', blue_header_format_bold)
        #SUBTOTAL PEDIDOS MN
        worksheet.write('I'+str(trow), total_mn, blue_content_bold)
        #SUBTOTAL PEDIDOS DLLS
        worksheet.write('J'+str(trow), total_dlls, blue_content_bold)
        #TOTAL POR COBRAR MN
        worksheet.write_formula('K'+str(trow),  '{=SUM(K'+str(trow-len(this_pedidos))+':K'+str(trow-1)+')}', blue_content_bold)
        #TOTAL POR COBRAR DLLS
        worksheet.write_formula('L'+str(trow),  '{=SUM(L'+str(trow-len(this_pedidos))+':L'+str(trow-1)+')}', blue_content_bold)
       
        worksheet.write('M'+str(trow+1),  "{:.2f}".format((this_pedidos['total'].sum()-cobros_dlls['amount'].sum()-cobros_mn['amount'].sum())*100/this_pedidos['total'].sum())+"%", blue_content_bold)
        #TOTAL POR FACTURAR DLLS
        worksheet.write_formula('N'+str(trow),  '{=SUM(N'+str(trow-len(this_pedidos))+':N'+str(trow-1)+')}',blue_content_bold)
        worksheet.write_formula('O'+str(trow),  '{=SUM(O'+str(trow-len(this_pedidos))+':O'+str(trow-1)+')}',blue_content_bold)


        #TOTALES TOTAL DE ADEBIS
        worksheet.write('H'+str(trow), 'Subtotales', blue_header_format_bold)
        worksheet.merge_range('I'+str(trow+1)+':J'+str(trow+1),' ',blue_content_bold)
        worksheet.write_formula('I'+str(trow+1)+':J'+str(trow+1),  'SUM(I'+str(trow)+'+J'+str(trow)+'*'+str(tc)+')',blue_content_bold)

        worksheet.merge_range('K'+str(trow+1)+':L'+str(trow+1),' ',blue_content_bold)
        worksheet.write_formula('K'+str(trow+1)+':L'+str(trow+1),  'SUM(K'+str(trow)+'+L'+str(trow)+'*'+str(tc)+')',blue_content_bold)

        worksheet.merge_range('N'+str(trow+1)+':O'+str(trow+1),' ',blue_content_bold)
        worksheet.write_formula('N'+str(trow+1)+':O'+str(trow+1),  'SUM(N'+str(trow)+'+O'+str(trow)+'*'+str(tc)+')',blue_content_bold)
        
        
        row_index=trow

# worksheet.write('K'+str(trow), str(cobros['amount'].sum()), blue_content)
# worksheet.write('L'+str(trow), str(cobros['exchange_sell'].values[0]*cobros['amount'].sum()), blue_content_bold)

worksheet.set_column('A:A',15)
worksheet.set_column('E:E',20)
worksheet.set_column('L:L',15)
worksheet.set_column('G:G',15)
worksheet.set_column('H:H',15)
worksheet.set_column('I:N',15)
worksheet.set_column('P:T',15)

#worksheet.set_landscape()
worksheet.set_paper(9)
worksheet.fit_to_pages(1, 1)  
workbook.close()

ModuleNotFoundError: No module named 'xlsxwriter'

In [18]:
 "{:.2f}".format((total_mn+total_dlls-(cobros_dlls['amount'].sum()+cobros_mn['amount'].sum()) )*100/pedidos.loc[pedidos['customer_id']==clientes['id'].values[i],'total'].sum())+"%"

'36.21%'

In [5]:
pedidos_dlls

,id,invoice,date,customer_id,seller_id,date_delivery,instalation_date,shipment,customer_shipping_address_id,coin_id,...,ret,tasa,description,category,clave,alias,exchange_sell,coin,symbol,code


In [14]:
(this_pedidos['total'].values[k]- this_cobros['amount'].sum())*100/this_pedidos['total'].values[k]

50.0

In [ ]:
base = pd.read_excel('Clientes.xls')

In [ ]:
base.columns

Index(['Clave', 'Estatus', 'Nombre', 'R.F.C.', 'Calle', 'Número interior',
       'Número exterior', 'Entre calle 1', 'Y calle', 'Colonia',
       'Código postal', 'Población', 'Municipio', 'Estado', 'País',
       'Nacionalidad', 'Referencia de la dirección', 'Teléfono',
       'Clasificación', 'Página web', 'C.U.R.P.', 'Clave de zona',
       'Enviar por correo electrónico', 'Mail predeterminado',
       'Día de revisión', 'Clave de vendedor', 'Tipo de empresa',
       'Régimen fiscal'],
      dtype='object')

In [ ]:
columnas

Index(['id', 'customer', 'customer_rfc', 'customer_state', 'customer_city',
       'customer_suburb', 'customer_street', 'customer_outdoor',
       'customer_indoor', 'customer_zip_code', 'customer_email',
       'customer_telephone', 'created_at', 'updated_at', 'legal_name', 'alias',
       'ncomp', 'clave', 'regimen_fiscal', 'pagina_web', 'curp',
       'nacionalidad'],
      dtype='object')

In [ ]:
base2=base[['Nombre', 'R.F.C.','Estado', 
       'Municipio', 'Colonia','Calle','Número exterior',  'Número interior',
       'Código postal','Mail predeterminado','Teléfono','Nombre','Nombre','Clave','Régimen fiscal',
        'Página web','C.U.R.P.','Nacionalidad']].fillna(0)
print(base2.columns)

Index(['Nombre', 'R.F.C.', 'Estado', 'Municipio', 'Colonia', 'Calle',
       'Número exterior', 'Número interior', 'Código postal',
       'Mail predeterminado', 'Teléfono', 'Nombre', 'Nombre', 'Clave',
       'Régimen fiscal', 'Página web', 'C.U.R.P.', 'Nacionalidad'],
      dtype='object')


In [ ]:
cols = "`,`".join([str(i) for i in columnas.tolist()])
print(cols)

id`,`customer`,`customer_rfc`,`customer_state`,`customer_city`,`customer_suburb`,`customer_street`,`customer_outdoor`,`customer_indoor`,`customer_zip_code`,`customer_email`,`customer_telephone`,`created_at`,`updated_at`,`legal_name`,`alias`,`ncomp


In [ ]:
116/1.16

100.0

In [ ]:
cursor=cnx.cursor(buffered=True)

In [ ]:
cursor.execute('select * from cobros')

In [ ]:
import pandas as pd

In [ ]:
cobros=pd.read_sql('select * from cobros',cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [2]:
import sys
import mysql.connector
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import os
from dotenv import load_dotenv
load_dotenv()
#ESTE ARGUMENTO NO SE USA EN ESTE REPORTE, SERÁ 0 SIEMPRE UWU
id=str(sys.argv[1])
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')

a_color='#354F84'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)


In [ ]:
cursor=cnx.cursor(buffered=True)

In [ ]:
cobros=cobros.fillna(0)
for i in range(0,len(cobros)):
    if(cobros['facture_id'].values[i]!=0):
        print(i)
        cursor.execute('insert into cobro_factures(cobro_id,facture_id) values('+str(cobros['id'].values[i])+','+str(cobros['facture_id'].values[i])+')')

0
1
2


In [ ]:
len(cobros)

4

In [ ]:
cnx.commit()

In [ ]:
import sys
import mysql.connector
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import os
from dotenv import load_dotenv
load_dotenv()
#ESTE ARGUMENTO NO SE USA EN ESTE REPORTE, SERÁ 0 SIEMPRE UWU
id=str(sys.argv[1])
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
a_color='#354F84'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
#Seccion para traer informacion de la base
query = ('SELECT * from customers where id = 1')

# join para cobros
# cobros=pd.read_sql('Select cobros.* ,customers.customer,internal_orders.invoice, users.name from ((cobros inner join internal_orders on internal_orders.id = cobros.order_id) inner join customers on customers.id = internal_orders.customer_id )inner join users on cobros.capturo=users.id',cnx)


#traer datos de los pedidos
pedidos=pd.read_sql("""Select internal_orders.* ,customers.clave,customers.alias,
coins.exchange_sell, coins.coin, coins.symbol
from ((
    internal_orders
    inner join customers on customers.id = internal_orders.customer_id )
    inner join coins on internal_orders.coin_id = coins.id)
     """,cnx)
cobros=pd.read_sql("""select cobro_orders.*
                     from (((
                         cobro_orders 
    inner join cobros on cobros.id=cobro_orders.cobro_id)
    inner join internal_orders on internal_orders.id = cobros.order_id )
    inner join coins on internal_orders.coin_id = coins.id) """,cnx)
facturas=pd.read_sql("""select * 
                     from ((
                         factures
    inner join internal_orders on internal_orders.id = factures.order_id )
    inner join coins on internal_orders.coin_id = coins.id) """,cnx)
creditos=pd.read_sql("""select * 
                     from ((
                         credit_notes    inner join internal_orders on internal_orders.id = credit_notes.order_id )
    inner join coins on internal_orders.coin_id = coins.id) """,cnx)
print(creditos)

Empty DataFrame
Columns: [id, customer_id, facture_id, amount, date, created_at, updated_at, status, credit_note, coin_id, order_id, id, invoice, date, customer_id, seller_id, date_delivery, instalation_date, shipment, customer_shipping_address_id, coin_id, subtotal, iva, total, payment_conditions, pays, observations, status, authorization_id, created_at, updated_at, comision, reg_date, oc, ncontrato, isr, ieps, descuento, ncotizacion, otra, dgi, noha, ret, tasa, description, category, id, coin, symbol, code, exchange_rate, date_application, created_at, updated_at, exchange_buy, exchange_sell]
Index: []

[0 rows x 56 columns]


C:\Users\hp\AppData\Local\Temp\ipykernel_19540\3827783321.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pedidos=pd.read_sql("""Select internal_orders.* ,customers.clave,customers.alias,
C:\Users\hp\AppData\Local\Temp\ipykernel_19540\3827783321.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cobros=pd.read_sql("""select cobro_orders.*
C:\Users\hp\AppData\Local\Temp\ipykernel_19540\3827783321.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  facturas=pd.read_sql("""select *
C:\Users\hp\AppData\Local\Temp\ipyker

In [ ]:
facturas.loc[facturas['order_id']==229]

,id,order_id,ordinal,facture,amount,status,npagos,created_at,updated_at,date,...,id,coin,symbol,code,exchange_rate,date_application,created_at,updated_at,exchange_buy,exchange_sell


In [5]:
#Filtrando datos---------------------------------------
#Separar pedidos vigentes
    #Agregar columna con saldo en los cobros
pedidos=pedidos.assign(saldo=0.0)
for i in range(len(pedidos)):
    pedidos['saldo'].values[i]=cobros.loc[cobros['order_id']==pedidos['id'].values[i],'amount'].sum()
pedidos=pedidos.loc[pedidos['total']-pedidos['saldo']>1]

C:\Users\hp\AppData\Local\Temp\ipykernel_20236\3652135030.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pedidos=pd.read_sql("""Select internal_orders.* ,customers.clave,customers.alias,
C:\Users\hp\AppData\Local\Temp\ipykernel_20236\3652135030.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  clientes=pd.read_sql("select * from customers",cnx)
C:\Users\hp\AppData\Local\Temp\ipykernel_20236\3652135030.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cobros=pd.read_sql("""select cobros.*,internal_orders.coin_id 

   id  customer_id facture_id     amount        date  created_at  updated_at  \
0   5          506       None  329200.00  2022-07-20  2023-05-02  2023-05-02   
1   6          394       None   49794.16  2023-05-31  2023-06-02  2023-06-02   
2   7          389       None  136870.00  2022-11-01  2023-06-07  2023-06-07   
3   8          407       None  202636.54  2023-05-09  2023-06-07  2023-06-07   
4   9          315       None  940070.84  2023-05-23  2023-06-12  2023-06-12   
5  10          312       None  464058.00  2023-02-17  2023-06-13  2023-06-13   
6  12          610       None  435598.56  2023-06-09  2023-06-23  2023-06-23   
7  14          601       None  903060.00  2023-07-08  2023-07-12  2023-07-12   

      status credit_note  order_id  coin_pedido  customer_pedido  
0  CAPTURADA      NC 309       116            1              506  
1  CAPTURADA      NC 357       206            1              394  
2  CAPTURADA      NC 320       135            1              389  
3  CAPTURAD

In [12]:
(pedidos['total']-pedidos['subtotal']*(1-pedidos['descuento'])*0.16).sum()

104747983.72

In [6]:
pedidos.loc[pedidos['invoice']=='2887']

,id,invoice,date,customer_id,seller_id,date_delivery,instalation_date,shipment,customer_shipping_address_id,coin_id,...,ret,tasa,description,category,clave,alias,exchange_sell,coin,symbol,code
24,147,2887,2023-04-25,414,3,2022-08-30,2022-08-30,Sí,53,1,...,0.0,0.0,Servicio directo SD,Servicios,8,FEDEX,1.0,NACIONAL,$,MN


In [13]:
cobros.loc[cobros['order_id']==147,'amount'].sum()/1.16

217000.00000000003

In [26]:
p=pedidos.loc[pedidos['coin']=='NACIONAL']

In [28]:
#TODO: calculado desde el dataframe cuanto dan los subtotales mn nacional?
p['total'].sum()

129170937.4448

In [12]:
111432232.78-111853526.41793102

-421293.63793101907

In [ ]:
sudo chown -R www-data: report
sudo chmod -R 755 report

In [ ]:
#Sin iva cuanto da?


In [24]:
len(p)

3

In [49]:
import sys
import mysql.connector
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import os
from dotenv import load_dotenv
load_dotenv()
#ESTE ARGUMENTO NO SE USA EN ESTE REPORTE, SERÁ 0 SIEMPRE UWU
id=str(sys.argv[1])
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')

a_color='#354F84'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
#Seccion para traer informacion de la base
query = ('SELECT * from customers where id =1')

# join para cobros
# cobros=pd.read_sql('Select cobros.* ,customers.customer,internal_orders.invoice, users.name from ((cobros inner join internal_orders on internal_orders.id = cobros.order_id) inner join customers on customers.id = internal_orders.customer_id )inner join users on cobros.capturo=users.id',cnx)


#traer datos de los pedidos
pedidos=pd.read_sql("""Select internal_orders.* ,customers.clave,customers.alias,
coins.exchange_sell, coins.coin, coins.symbol,coins.code
from ((
    internal_orders
    inner join customers on customers.id = internal_orders.customer_id )
    inner join coins on internal_orders.coin_id = coins.id)
     """,cnx)
clientes=pd.read_sql("select * from customers",cnx)
cobros=pd.read_sql("""select cobros.*,internal_orders.coin_id as coin_pedido,internal_orders.customer_id  
                     from (
                         cobros
    inner join internal_orders on internal_orders.id = cobros.order_id )""",cnx)
facturas=pd.read_sql("""select factures.*,internal_orders.coin_id as coin_pedido,internal_orders.customer_id  
                     from (
                         factures
    inner join internal_orders on internal_orders.id = factures.order_id )""",cnx)
creditos=pd.read_sql("""select credit_notes.*,internal_orders.coin_id as coin_pedido,internal_orders.customer_id as customer_pedido 
                     from (
                         credit_notes
    inner join internal_orders on internal_orders.id = credit_notes.order_id ) """,cnx)

nordenes=len(pedidos)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site

In [50]:
#Separar pedidos vigentes
    #Agregar columna con saldo en los cobros
pedidos=pedidos.assign(saldo=0.0)
for i in range(len(pedidos)):
    pedidos['saldo'].values[i]=cobros.loc[cobros['order_id']==pedidos['id'].values[i],'amount'].sum()

In [19]:
# ej, el primer pedido
pedidos[['invoice','total','saldo']]

,invoice,total,saldo
0,2175,3.561200e+05,356120.00
1,2749,3.190000e+03,3190.00
2,2750,5.308694e+07,49725362.80
3,2751,5.800000e+03,0.00
4,2752,8.288003e+04,82880.03
...,...,...,...
189,3104,3.157868e+06,0.00
190,3105,8.827600e+04,0.00
191,3106,1.118471e+06,0.00
192,3107,2.298192e+05,114909.60


In [20]:
pedidos['total']-pedidos['saldo']

0            0.000
1            0.000
2      3361573.200
3         5800.000
4           -0.002
          ...     
189    3157868.000
190      88276.000
191    1118470.840
192     114909.600
193    1127520.000
Length: 194, dtype: float64

In [23]:
len(pedidos)

194

In [51]:
pedidos=pedidos.loc[pedidos['total']-pedidos['saldo']<=100]

In [52]:
len(pedidos)

134

In [27]:
pedidos=pedidos.assign(pendiente=pedidos['total']-pedidos['saldo'])

In [32]:
pedidos['pendiente'].unique()

array([ 0.00000000e+00, -1.99999999e-03,  1.20000006e-03, -1.50000000e-01,
       -2.14959600e+04, -8.00000000e-02, -1.00000000e-02,  1.80000000e-01,
        2.79999999e-03, -1.99999986e-03, -7.00000001e-02,  8.00000000e-02,
       -4.80000000e-01,  2.00000000e-03,  4.80000000e-03,  3.19999992e-03,
       -2.80000000e-03,  5.82076609e-11,  2.32830644e-10, -4.80000001e-03,
       -9.99999998e-03, -2.91038305e-11,  2.91038305e-11,  4.80000002e-03,
       -4.99999999e-02,  9.00000000e+01])

In [43]:
pedidos.loc[pedidos['pendiente']>1,['invoice','customer_id','code','total','saldo','pendiente']]

,invoice,customer_id,code,total,saldo,pendiente
157,3072,645,MN,83056.0,82966.0,90.0


In [39]:
clientes[clientes['id']==645]

,id,customer,customer_rfc,customer_state,customer_city,customer_suburb,customer_street,customer_outdoor,customer_indoor,customer_zip_code,...,created_at,updated_at,legal_name,alias,ncomp,clave,regimen_fiscal,pagina_web,curp,nacionalidad
341,645,COMERCIALIZADORA JAVA 2011,CJD110214H29,MEXICO,CDMX,LOMAS DE PADIERNA SUR,PROLONGACION TULUM,MZ 49,LT 23,14740,...,2023-08-18 08:12:41,2023-08-18 08:12:41,S.A. DE C.V.,JAVA 2011,None,350,None,None,None,None


In [1]:
import sys
import mysql.connector
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import os
from dotenv import load_dotenv
load_dotenv()
#ESTE ARGUMENTO NO SE USA EN ESTE REPORTE, SERÁ 0 SIEMPRE UWU
id=str(sys.argv[1])
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')

a_color='#354F84'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
#Seccion para traer informacion de la base
query = ('SELECT * from customers where id =1')


In [ ]:
curso

In [21]:
clientes

,id,customer,customer_rfc,customer_state,customer_city,customer_suburb,customer_street,customer_outdoor,customer_indoor,customer_zip_code,...,created_at,updated_at,legal_name,alias,ncomp,clave,regimen_fiscal,pagina_web,curp,nacionalidad
0,297,GALLETAS DE CALIDAD S.A.de C.V.,GCA690815R94,CIUDAD DE MEXICO,AZCAPOTZALCO,COLTONGO,PONIENTE 122,489,0,2630.,...,NaT,NaT,POR ASIGNAR,GALLETAS DE CALIDAD S.A.de C.V.,None,179,0.0,http://www.marian.com.mx/web/,0,MEXICANA
1,298,CASA DIAZ MAQUINAS DE COCER S.A. DE C.V.,CDM760124C85,CD DE MEXICO,CUCUTHEMOC,OBRERA,FRAY SERVANDO TERESA DE MIER,29,0,6800.,...,NaT,NaT,POR ASIGNAR,CASA DIAZ MAQUINAS DE COCER S.A. DE C.V.,None,191,0.0,CASADIAZ..COM.MX,0,MEXICANA
2,299,LO,0,0,0,0,0,0,0,0.0,...,NaT,NaT,POR ASIGNAR,LO,None,160,0.0,0,0,0
3,300,FINANMADRID MEXICO,FME050125MS0,CIUDAD DE MEXICO,MIGUEL HIDALGO,LOMAS DE CHAPULTEPEC 1 SECCION,BLVD MANUEL AVILA CAMACHO,P 4 Y 5,0,11000,...,NaT,NaT,POR ASIGNAR,FINANMADRID MEXICO,None,161,0.0,0,0,0
4,301,COMERCIALIZADORA KELLY S.A. DE C.V.,CKE051019KG1,MEXICO,NAUCALPAN DE JUAREZ,LOMAS DE TECAMACHALCO SECCION FUENTES,FUENTES DE PIRAMIDES,1,604,53950,...,NaT,NaT,POR ASIGNAR,COMERCIALIZADORA KELLY S.A. DE C.V.,None,119,0.0,0,0,MEXICANA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,655,MAS BODEGA Y LOGISTICA,CCA030408D46,NUEVO LEON,NUEVO LEON,PARQUE INDUSTRIAL KALOS,CARR CUOTA MONTERREY REYNOSA,9650,a,67110,...,2023-10-05 10:32:08,2023-10-05 10:32:08,S.A. DE C.V.,MAS BODEGA Y LOGISTICA,None,359,None,None,None,None
352,656,PASE LOGISTICA,PLO220308375,MEXICO,CDMX,INDUSTRIAL VALLEJO,PONIENTE 146,603,None,02300,...,2023-10-10 04:38:52,2023-10-10 04:38:52,S.A. DE C.V.,PASE,None,360,None,None,None,None
353,657,EXCEL NOBLEZA,ENO940203NY2,PUEBLA,PUEBLA,TEPANCO DE LOPEZ,CARR FEDERAL PUEBLA-TEHUACAN,101,None,75800,...,2023-10-20 11:06:38,2023-10-20 11:06:38,S DE R.L DE C.V.,EXCEL NOBLEZA,None,361,None,None,None,None
354,658,CROWN GAMING MEXICO,CGM100309IL8,MEXICO,CDMX,POLANCO,MONTE ELBRUZ,132,804,11530,...,2023-10-24 05:34:53,2023-10-24 05:34:53,FISICA CAEYP,CROWN GAMING MEXICO,None,364,None,None,None,None


In [54]:
cobros=cobros.merge(pedidos[['id','code']].rename(columns={'id':'order_id'}),how='inner',on='order_id')

In [55]:
len(cobros)

293

In [62]:
cobros.loc[cobros['code']=='MN','amount'].sum()+tc*cobros.loc[cobros['code']!='MN','amount'].sum()

NameError: name 'tc' is not defined

In [4]:
!pip install pandas


[notice] A new release of pip available: 22.2.2 -> 23.3.2


[notice] To update, run: C:\Python310\python.exe -m pip install --upgrade pip


In [2]:
import sys
import mysql.connector
import pandas as pd
import sys
import mysql.connector
import os
from dotenv import load_dotenv
load_dotenv()
#ESTE ARGUMENTO NO SE USA EN ESTE REPORTE, SERÁ 0 SIEMPRE UWU
id=0
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')

a_color='#354F84'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)

In [3]:
cursor=cnx.cursor(buffered=True)

In [8]:
gastos=pd.read_excel('GASTOS.xlsx',skiprows=[0,1,2])

In [9]:
gastos

,Unnamed: 0,PDA,DESCRIPCION,TOTALES,ENERO,Unnamed: 5,Unnamed: 6,Unnamed: 7,FEBRERO,Unnamed: 9,...,Unnamed: 52,TOTAL AÑO 2023,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,PROMEDIO MENSUAL PARA 2023,Unnamed: 59,Unnamed: 60,Unnamed: 61
0,NaN,NaN,NaN,PRESUPUESTADOS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,SIN IVA (PROMEDIO),CORPORATIVO,PLANTA,TOTAL,% DEL TOTAL,CORPORATIVO,PLANTA,...,NaN,CORPORATIVO,PLANTA,TOTAL,% DEL TOTAL,NaN,CORPORATIVO,PLANTA,TOTAL,% DEL TOTAL
2,NaN,1,RENTA CORPORATIVO,1,1,1,$2.00,1.75%,1,1,...,NaN,$12.00,$12.00,$24.00,1.75%,NaN,$1.00,$1.00,$2.00,1.75%
3,NaN,2,RENTA PLANTA XAHUENTO,1,1,1,$2.00,1.75%,1,1,...,NaN,$12.00,$12.00,$24.00,1.75%,NaN,$1.00,$1.00,$2.00,1.75%
4,NaN,3,AGUA,1,1,1,$2.00,1.75%,1,1,...,NaN,$12.00,$12.00,$24.00,1.75%,NaN,$1.00,$1.00,$2.00,1.75%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,NaN,6,PRODUCTOS DE ASEO,NaN,$8.00,$8.00,$8.00,$8.00,$8.00,$8.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,NaN,7,"Vo. Bo., LICENCIAS, VERIFICACIONES Y SIMILARES",NaN,$14.00,$14.00,$14.00,$14.00,$14.00,$14.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,NaN,8,"GASOLINA, PEAJES, VIATICOS.",NaN,$12.00,$12.00,$12.00,$12.00,$12.00,$12.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,NaN,NaN,NaN,TOTALES,$108.00,$108.00,$108.00,$108.00,$108.00,$108.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
gastos['DESCRIPCION']=gastos['DESCRIPCION'].fillna('')

In [13]:
import numpy as np

In [28]:
for i in range(0,59):
    if(len(gastos['DESCRIPCION'].values[i])>3):
        print(gastos['DESCRIPCION'].values[i])
        cursor.execute("insert into bills(description , p_total) values('"+str(gastos['DESCRIPCION'].values[i])+"', 1)")

RENTA CORPORATIVO 
RENTA PLANTA XAHUENTO
AGUA
CFE (ENERGIA ELECTRICA)
TELEFONOS (FIJOS + MOVILES)
GAS PARA HORNO DE PINTURA
RECOLECCION BASURA
SALARIOS
HORAS EXTRAS Y OTRAS PERCEPCIONES POR SALARIOS
FINIQUITOS, INDEMNIZACIONES Y SIMILARES.
AGUINALDOS (PROVISION).
VACACIONES (PROVISION).
PRIMA VACACIONAL (PROVISION)
INCENTIVOS POR PRODUCTIVIDAD
IMSS
INFONAVIT
IMPUESTOS SOBRE NOMINAS (3%)
PTU (PROVISION)
IMPUESTO SOBRE EMISIONES A LA ATMOSFERA
OTROS IMPUESTOS
DGI NOMINA (COMISIONES)
DGI FEFE
HONORARIOS CONTADORES Y AUDITORES
HONORARIOS ASESOR INFORMATICO Y T.I. (TINORED)
HONORARIOS ABOGADO (ASESORIA JURIDICA)
MERCADOTECNIA Y PUBLICIDAD
COACHING
ASPEL (SAE, NOI, COI)
SERVICIO DE RECLUTAMIENTO Y RECURSOS HUMANOS
CAPACITACION EXTERNA
VIGILANCIA / SERVICIO
LIMPIEZA / SERVICIO
MANTENIMIENTO  DEL CORPORATIVO
MANTENIMIENTO  PLANTA (OFICINAS)
MANTENIMIENTO  (SISTEMAS DE INFORMACION)
MANTENIMIENTO VEHICULOS
SEGUROS DE VEHICULOS 
SEGUROS DE RESPONSABILIDAD CIVIL Y OTROS
PRODUCTOS ASEO
PAPELERIA
GA

In [29]:
cnx.commit()

In [6]:
reg

,clave,desc,fisica,moral,Unnamed: 4
0,601,General de Ley Personas Morales,No,Sí,2022-01-01
1,603,Personas Morales con Fines no Lucrativos,No,Sí,2022-01-01
2,605,Sueldos y Salarios e Ingresos Asimilados a Sal...,Sí,No,2022-01-01
3,606,Arrendamiento,Sí,No,2022-01-01
4,607,Régimen de Enajenación o Adquisición de Bienes,Sí,No,2022-01-01
5,608,Demás ingresos,Sí,No,2022-01-01
6,610,Residentes en el Extranjero sin Establecimient...,Sí,Sí,2022-01-01
7,611,Ingresos por Dividendos (socios y accionistas),Sí,No,2022-01-01
8,612,Personas Físicas con Actividades Empresariales...,Sí,No,2022-01-01
9,614,Ingresos por intereses,Sí,No,2022-01-01


In [1]:
# migrar clientes
import sys
import mysql.connector
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import os
from dotenv import load_dotenv
load_dotenv()
#ESTE ARGUMENTO NO SE USA EN ESTE REPORTE, SERÁ 0 SIEMPRE UWU
id=0
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')

a_color='#354F84'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
#Seccion para traer informacion de la base
customers_columns=pd.read_sql('select * from customers limit 1',cnx).columns


C:\Users\hp\AppData\Local\Temp\ipykernel_3588\1154115680.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customers_columns=pd.read_sql('select * from customers limit 1',cnx).columns


In [2]:
data=pd.read_excel('CLIENTES SICAR 1 (1) (1).xlsx',skiprows=[0])

In [3]:
xl = pd.ExcelFile('CLIENTES SICAR 1 (1) (1).xlsx')
# xl.sheet_names  # see all sheet names


In [5]:
cursor=cnx.cursor(buffered=True)

In [10]:
data['Celular']= data['Celular'].fillna('')

In [15]:
for sheet in xl.sheet_names:
    print(sheet)
    data=pd.read_excel('CLIENTES SICAR 1 (1) (1).xlsx',sheet_name=sheet,skiprows=[0])
    data['Celular']= data['Celular'].fillna(0)
    for i in range(0,len(data)):
        if(data['Celular'].values[i]==0):
            cel=' '
        else:
            try:
                cel=str(int(data['Celular'].values[i]))
            except:
                cel=' '
        cursor.execute(f"insert into customers (customer,alias,clave,customer_telephone,mobil) values ('{data['Nombre'].values[i]}','{data['Nombre'].values[i]}','{data['No. Cli'].values[i]}','{data['Teléfono'].values[i]}','{cel}')")
    

Table 1
Table 2
Table 3
Table 4
Table 5
Table 6
Table 7
Table 8
Table 9
Table 10
Table 11
Table 12
Table 13
Table 14
Table 15
Table 16
Table 17
Table 18
Table 19
Table 20
Table 21
Table 22
Table 23
Table 24
Table 25
Table 26
Table 27
Table 28
Table 29
Table 30
Table 31
Table 32
Table 33
Table 34
Table 35
Table 36
Table 37
Table 38
Table 39
Table 40
Table 41
Table 42
Table 43
Table 44
Table 45


In [1]:
print(x)

NameError: name 'x' is not defined

In [16]:
cnx.commit()